### Atoms

AtomBuilder3 (AB3) builds charge field atomic models selected by 
the user. AB3 is a copy of mBuilder, utilizing a datafile, and one 
and two directional proton configuration slot groups replacing earlier 
versions, AtomBuilder (AB) and AtomBuilder2 (AB2). AB and AB2 are now 
in the additional notebooks folder as history files. mbuilder carries 
on as the day-to-day working program intended to develop a 3D charge 
field model rendering of molecular matter selected by the user.  

Operating instructions. Start by clicking on the "Atom data" cell,
about 10 cells below. Next, from the toolbar's Cell dropdown selection 
menu choose "Run All Above". When the gui in the cell above "Atom data" 
becomes visible, make any changes to it or select an atom from the 
Periodic table. Next, with the gui or the Periodic table cell still
active, select "Run All Below" from the Cell dropdown menu. You'll then 
view the main program outputs from the tab enclosure cell near the 
bottom. For any changes, repeat - go back to the gui or Periodic 
table, make changes, then select "Run All Below". These instructions 
are repeated in the "Atom data" cell.

Atoms is intended to introduce Charge field theory - see the 
notebook's end cell's source references.                                          

In [1]:
# The imports necessary to create the 3D modeling 
# and gui controls.
from pythreejs import *
from IPython.display import display
from ipywidgets import HBox, VBox, Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan, copysign
from random import random , randint , seed

In [2]:
# A few values used
phi = (1+sqrt(5))/2  # calculating phi-based-pi, piG.
piG = 4/sqrt(phi)    # piG is used instead of pi.
pRadius = 0.05 # proton radius, used by all atomic distances
caroSecs = 5.0 # Problem Here # time in secs for the carousel to make one rotation

In [3]:
# For pythreejs display, sizing the view, camera and adding lights.
view_width = 600
view_height = 400
camera = CombinedCamera(position=[0, 0, 10], width=view_width, height=view_height)
# See renderer cell for info, non-jason compliant pythreejs orbital camera user warning 
key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [4]:
# For working with a dataFrame.
import pandas as pd
import numpy as np

# Sizing the users' local display of tabular data if needed.
#pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)

# Don't read the slotLayout diags from Cr6-ElementsdataFrame,
# this programm creates its own SL diagram.
#filename = 'data/Cr6-Elements.csv'   
filename = 'data/Cr6-Elements-M.csv' # changing slot configurations 
df_raw = pd.read_csv(filename) 
#df_raw = df_raw.loc[:,:('CanBind')]
# Change. The SL diags from Cr6-Elements are read but not used
df_raw = df_raw.loc[:,:('SL4')] 

# Uncomment df_raw or df_raw.info() to view
# df_raw
# df_raw.info()

# Make changes to the dataFrame created by pd.read_csv(filename)'s 
# method defaults. The data type constrains how you may work with 
# the data.
df = df_raw.copy()
# Columns "SlotLayout" through "SL4" contain many non-value entries
# that pandas interprets as NaNs. Convert the NaNs to blank spaces.
df = df.fillna(" ")  

# Duplicating "AtomicNumber", one as a string,the other an int.
df["AtomicSlotGrp"] = df["AtomicNumber"].copy()
# df["AtomicNumber"] defaults to an integer. Make it string.
df["AtomicNumber"] = df["AtomicNumber"].astype(str)

# Make "CanBind" boolean.
df["CanBind"] = np.where(df["CanBind"] == "Y", True, False)

# Proton stack locatios need to be float
df["PX"] = df["PX"].astype("float")
df["PY"] = df["PY"].astype("float")
df["PZ"] = df["PZ"].astype("float")
df["P2P3"] = df["P2P3"].astype("float")
df["P12"] = df["P12"].astype("float")
df["PE"] = df["PE"].astype("float")

# Uncomment df or df.info() to view
#df_raw
#df
#df.info()

In [5]:
"""
Elements and Molecules 

'elements'is a list of tuples used by the program to  
identify or re-select a desired atom from the gui 
dropdown or periodic table. Also used as the optional
3D atom's atomic label. 'elements' is taken from the 
df['AtomicTuple'] column.  

Starting with 1710 rows. After drop_duplicates, the 
'AtomicTuple' row index increments by 19's, 0 to 1709, 
19x expanded. The Series.array method renumbers the 
index back to the desired 0-89. The two steps are 
chained.

Next, atomicTups has been read as a list of strings. 
Those strings are converted into tuples by using 
the map and eval methods.
"""
atomicTups = df['AtomicTuple'].drop_duplicates().array
elements = list(map(eval, atomicTups))
#elements

# Molecules. A user will select the 'molecule': 
# numberM (1-4) from this set.
molecules = {'H2':1,'C2':2,'N2':3,'O2':4}
#molecules

# Each triplet below (in numberN position) incl: 1. total num of  
# atoms; 2. the atomic number for the first; and 2. second atom.
moleConfig = [(2,1,1), (2,6,6), (2,7,7), (2,8,8)]
#moleConfig

In [6]:
# The periodic table's 90 buttons. Each button  
# includes: 1. Atomic type color, clist, worked 
# in steps first-fifth. 2. Atomic symbol, dlist. 
# 3. Layout width and layout height

# First, the ten Atomic types, tenATypes. 
# Note. tenATypes = list(df['Atomic ...duplicates())
# works ok, the explicit written list is simpler). 
tenATypes = []
tenATypes = list(df['AtomicType'].drop_duplicates())  
#tenATypes = ['nonmetal','noble_gas','alkali_metal','alkaline_earth','metalloid',
#             'halogen','basic_metal','transition_metal','lanthanide','actinide']
#tenATypes

# Second - The 10 atomic type colors, tenColors. 
# Lined up with tenATypes
tenColors = ['skyblue','cyan','tomato','orange','springgreen','powderblue','palegreen',
         'yellow','thistle','violet']

# Third, zip together the type/color dictionary
typeColorDict = {}
typeColorDict = dict(zip(tenATypes, tenColors))
#typeColorDict

# Fourth - creating a 1710 row number/type tuple
# list which is reduced to 90 unique rows.
df['allATypes'] = list(zip(df.AtomicNumber, df.AtomicType))
ATypes = df['allATypes'].drop_duplicates().array 
#ATypes

# Fifth - Build the list of 90 colors, clist,
# used to color the Periodic table's 90 buttons.
clist = []
for i in range(90):
    clist.append(typeColorDict[ATypes[i][1]])
#clist

# The atomic symbols for the Periodic table. 
dlist = df['AtomicSymbol'].drop_duplicates().array

# Defining the buttons by adding the atomic symbols and colors to the buttons
a1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
a2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
a3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
a4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
a5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
a6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
a7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
a8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
a9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
a10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
a11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
a12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
a13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
a14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
a15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
a16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
a17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
a18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
a19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
a20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
a21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
a22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
a23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
a24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
a25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
a26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
a27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
a28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
a29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
a30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
a31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
a32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
a33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
a34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
a35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
a36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
a37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
a38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
a39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
a40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
a41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
a42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
a43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
a44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
a45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
a46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
a47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
a48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
a49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
a50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
a51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
a52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
a53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
a54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
a55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
a56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
a57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
a58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
a59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
a60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
a61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
a62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
a63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
a64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
a65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
a66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
a67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
a68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
a69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
a70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
a71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
a72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
a73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
a74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
a75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
a76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
a77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
a78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
a79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
a80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
a81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
a82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
a83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
a84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
a85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
a86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
a87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
a88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
a89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
a90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
abttns = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,
               a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,
               a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,
               a60,a61,a62,a63,a64,a65,a66,a67,a68,a69,a70,a71,a72,a73,a74,a75,a76,a77,a78,a79,
               a80,a81,a82,a83,a84,a85,a86,a87,a88,a89,a90]
# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    abttns[i].layout = widgets.Layout(border = '1px solid black') 
    abttns[i].layout.width = '100%'
    abttns[i].layout.height = '100%'  #'40px'
    i += 1
    
# Layout the 90 buttons in periodic table form
grid = GridspecLayout(10, 18)

grid[0,0] = abttns[0]
grid[0,17] = abttns[1]
grid[1,0] = abttns[2]
grid[1,1] = abttns[3]
p=12
while p < 18:
    grid[1,p] = abttns[p-8]
    p += 1
grid[2,0] = abttns[10]
grid[2,1] = abttns[11]
n=12
while n < 18:
    grid[2,n] = abttns[n]
    n += 1
m=0
while m < 18:
    grid[3,m] = abttns[18+m]
    m += 1
k=0
while k < 18:
    grid[4,k] = abttns[36+k]
    k += 1
grid[5,0] = abttns[54]
grid[5,1] = abttns[55]
# grid[5,2] = the Lanthanide series hole
i=3
while i < 18:
    grid[5,i] = abttns[68+i]
    i += 1
grid[6,0] = abttns[86]
grid[6,1] = abttns[87]
# grid[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    grid[7,j] = abttns[54+j]
    j += 1
#Actinide series
grid[8,2] = abttns[88]
grid[8,3] = abttns[89]

# The periodic table button functions
output = widgets.Output()
#display(output)
@output.capture()

def on_a1_clicked(a1): selectAtom.value = 1 
def on_a2_clicked(a2): selectAtom.value = 2 
def on_a3_clicked(a3): selectAtom.value = 3 
def on_a4_clicked(a4): selectAtom.value = 4 
def on_a5_clicked(a5): selectAtom.value = 5 
def on_a6_clicked(a6): selectAtom.value = 6 
def on_a7_clicked(a7): selectAtom.value = 7 
def on_a8_clicked(a8): selectAtom.value = 8 
def on_a9_clicked(a9): selectAtom.value = 9 
def on_a10_clicked(a10): selectAtom.value = 10 
def on_a11_clicked(a11): selectAtom.value = 11 
def on_a12_clicked(a12): selectAtom.value = 12
def on_a13_clicked(a13): selectAtom.value = 13
def on_a14_clicked(a14): selectAtom.value = 14 
def on_a15_clicked(a15): selectAtom.value = 15 
def on_a16_clicked(a16): selectAtom.value = 16 
def on_a17_clicked(a17): selectAtom.value = 17 
def on_a18_clicked(a18): selectAtom.value = 18 
def on_a19_clicked(a19): selectAtom.value = 19 
def on_a20_clicked(a20): selectAtom.value = 20 
def on_a21_clicked(a21): selectAtom.value = 21 
def on_a22_clicked(a22): selectAtom.value = 22 
def on_a23_clicked(a23): selectAtom.value = 23 
def on_a24_clicked(a24): selectAtom.value = 24
def on_a25_clicked(a25): selectAtom.value = 25 
def on_a26_clicked(a26): selectAtom.value = 26
def on_a27_clicked(a27): selectAtom.value = 27
def on_a28_clicked(a28): selectAtom.value = 28
def on_a29_clicked(a29): selectAtom.value = 29 
def on_a30_clicked(a30): selectAtom.value = 30
def on_a31_clicked(a31): selectAtom.value = 31 
def on_a32_clicked(a32): selectAtom.value = 32
def on_a33_clicked(a33): selectAtom.value = 33
def on_a34_clicked(a34): selectAtom.value = 34
def on_a35_clicked(a35): selectAtom.value = 35
def on_a36_clicked(a36): selectAtom.value = 36
def on_a37_clicked(a37): selectAtom.value = 37
def on_a38_clicked(a38): selectAtom.value = 38
def on_a39_clicked(a39): selectAtom.value = 39
def on_a40_clicked(a40): selectAtom.value = 40 
def on_a41_clicked(a41): selectAtom.value = 41 
def on_a42_clicked(a42): selectAtom.value = 42
def on_a43_clicked(a43): selectAtom.value = 43
def on_a44_clicked(a44): selectAtom.value = 44 
def on_a45_clicked(a45): selectAtom.value = 45 
def on_a46_clicked(a46): selectAtom.value = 46 
def on_a47_clicked(a47): selectAtom.value = 47 
def on_a48_clicked(a48): selectAtom.value = 48 
def on_a49_clicked(a49): selectAtom.value = 49
def on_a50_clicked(a50): selectAtom.value = 50
def on_a51_clicked(a51): selectAtom.value = 51 
def on_a52_clicked(a52): selectAtom.value = 52 
def on_a53_clicked(a53): selectAtom.value = 53 
def on_a54_clicked(a54): selectAtom.value = 54 
def on_a55_clicked(a55): selectAtom.value = 55 
def on_a56_clicked(a56): selectAtom.value = 56 
def on_a57_clicked(a57): selectAtom.value = 57 
def on_a58_clicked(a58): selectAtom.value = 58 
def on_a59_clicked(a59): selectAtom.value = 59 
def on_a60_clicked(a59): selectAtom.value = 60 
def on_a61_clicked(a61): selectAtom.value = 61 
def on_a62_clicked(a62): selectAtom.value = 62 
def on_a63_clicked(a63): selectAtom.value = 63 
def on_a64_clicked(a64): selectAtom.value = 64 
def on_a65_clicked(a65): selectAtom.value = 65 
def on_a66_clicked(a66): selectAtom.value = 66 
def on_a67_clicked(a67): selectAtom.value = 67 
def on_a68_clicked(a68): selectAtom.value = 68 
def on_a69_clicked(a69): selectAtom.value = 69 
def on_a70_clicked(a70): selectAtom.value = 70 
def on_a71_clicked(a71): selectAtom.value = 71 
def on_a72_clicked(a72): selectAtom.value = 72 
def on_a73_clicked(a73): selectAtom.value = 73 
def on_a74_clicked(a74): selectAtom.value = 74 
def on_a75_clicked(a75): selectAtom.value = 75 
def on_a76_clicked(a76): selectAtom.value = 76 
def on_a77_clicked(a77): selectAtom.value = 77 
def on_a78_clicked(a78): selectAtom.value = 78 
def on_a79_clicked(a79): selectAtom.value = 79 
def on_a80_clicked(a80): selectAtom.value = 80 
def on_a81_clicked(a81): selectAtom.value = 81 
def on_a82_clicked(a82): selectAtom.value = 82 
def on_a83_clicked(a83): selectAtom.value = 83 
def on_a84_clicked(a84): selectAtom.value = 84 
def on_a85_clicked(a85): selectAtom.value = 85 
def on_a86_clicked(a86): selectAtom.value = 86 
def on_a87_clicked(a87): selectAtom.value = 87 
def on_a88_clicked(a88): selectAtom.value = 88 
def on_a89_clicked(a89): selectAtom.value = 89 
def on_a90_clicked(a90): selectAtom.value = 90     
    
a1.on_click(on_a1_clicked)
a2.on_click(on_a2_clicked)
a3.on_click(on_a3_clicked)
a4.on_click(on_a4_clicked)
a5.on_click(on_a5_clicked)
a6.on_click(on_a6_clicked)
a7.on_click(on_a7_clicked)
a8.on_click(on_a8_clicked)
a9.on_click(on_a9_clicked)
a10.on_click(on_a10_clicked)
a11.on_click(on_a11_clicked)
a12.on_click(on_a12_clicked)
a13.on_click(on_a13_clicked)
a14.on_click(on_a14_clicked)
a15.on_click(on_a15_clicked)
a16.on_click(on_a16_clicked)
a17.on_click(on_a17_clicked)
a18.on_click(on_a18_clicked)
a19.on_click(on_a19_clicked)
a20.on_click(on_a20_clicked)
a21.on_click(on_a21_clicked)
a22.on_click(on_a22_clicked)
a23.on_click(on_a23_clicked)
a24.on_click(on_a24_clicked)
a25.on_click(on_a25_clicked)
a26.on_click(on_a26_clicked)
a27.on_click(on_a27_clicked)
a28.on_click(on_a28_clicked)
a29.on_click(on_a29_clicked)
a30.on_click(on_a30_clicked)
a31.on_click(on_a31_clicked)
a32.on_click(on_a32_clicked)
a33.on_click(on_a33_clicked)
a34.on_click(on_a34_clicked)
a35.on_click(on_a35_clicked)
a36.on_click(on_a36_clicked)
a37.on_click(on_a37_clicked)
a38.on_click(on_a38_clicked)
a39.on_click(on_a39_clicked)
a40.on_click(on_a40_clicked)
a41.on_click(on_a41_clicked)
a42.on_click(on_a42_clicked)
a43.on_click(on_a43_clicked)
a44.on_click(on_a44_clicked)
a45.on_click(on_a45_clicked)
a46.on_click(on_a46_clicked)
a47.on_click(on_a47_clicked)
a48.on_click(on_a48_clicked)
a49.on_click(on_a49_clicked)
a50.on_click(on_a50_clicked)
a51.on_click(on_a51_clicked)
a52.on_click(on_a52_clicked)
a53.on_click(on_a53_clicked)
a54.on_click(on_a54_clicked)
a55.on_click(on_a55_clicked)
a56.on_click(on_a56_clicked)
a57.on_click(on_a57_clicked)
a58.on_click(on_a58_clicked)
a59.on_click(on_a59_clicked)
a60.on_click(on_a60_clicked)
a61.on_click(on_a61_clicked)
a62.on_click(on_a62_clicked)
a63.on_click(on_a63_clicked)
a64.on_click(on_a64_clicked)
a65.on_click(on_a65_clicked)
a66.on_click(on_a66_clicked)
a67.on_click(on_a67_clicked)
a68.on_click(on_a68_clicked)
a69.on_click(on_a69_clicked)
a70.on_click(on_a70_clicked)
a71.on_click(on_a71_clicked)
a72.on_click(on_a72_clicked)
a73.on_click(on_a73_clicked)
a74.on_click(on_a74_clicked)
a75.on_click(on_a75_clicked)
a76.on_click(on_a76_clicked)
a77.on_click(on_a77_clicked)
a78.on_click(on_a78_clicked)
a79.on_click(on_a79_clicked)
a80.on_click(on_a80_clicked)
a81.on_click(on_a81_clicked)
a82.on_click(on_a82_clicked)
a83.on_click(on_a83_clicked)
a84.on_click(on_a84_clicked)
a85.on_click(on_a85_clicked)
a86.on_click(on_a86_clicked)
a87.on_click(on_a87_clicked)
a88.on_click(on_a88_clicked)
a89.on_click(on_a89_clicked)
a90.on_click(on_a90_clicked)


In [7]:
""" Defining the gui widgets """
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emissionType= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
includeNeutrons = widgets.Checkbox(
    value=True,
    description='Show neutrons',
    disabled=False,
    indent=False
)
coloredEmissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the equatorial(or cone) emission radius
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EmRadius',
)
# Changes the distance between adjacent orthogonal protons
ePSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 7.0,
    step= 0.1,
    description='Equ1_Pole2',
)
# Changes the distance tween top and bottom alpha parallel protons
pPSlider = widgets.FloatSlider(
    value = 4.0,
    min = 3.0,
    max = 6.0,
    step= 0.1,
    description='A width',
)
# Changes the pole to pole distance between 2 adjacent alphas
p2p3Slider = widgets.FloatSlider(
    value = 3.0,
    min = 2.0,
    max = 5.0,
    step= 0.1,
    description='A1_A2 Sep',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)
cameraType= widgets.Dropdown(
    options=['orthographic', 'perspective'],
    value='perspective', 
    description='View:',
)

# The guiwidget control panel 
guiwidgets = widgets.VBox([selectAtom,atomsLabel,emissionType,
        coloredEmissions,includeNeutrons,emRSlider,ePSlider,
        pPSlider,p2p3Slider,coneAngleSlider,cameraType])

# Observe any changes in widget values, button or gui
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

displayNeutrons = includeNeutrons.value
def handle_change(change):
    global displayNeutrons 
    displayNeutrons = change.new
includeNeutrons.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

ortho = ePSlider.value # proton1 equator to proton2 pole
def handle_change(change):
    global ortho
    ortho = change.new
ePSlider.observe(handle_change, 'value');

pPTP = pPSlider.value  # proton pole to proton pole 
def handle_change(change):
    global pPTP
    pPTP = change.new
pPSlider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between 2 adjacent alphas
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

cAngleDeg = coneAngleSlider.value # cone angle
def handle_change(change):
    global cAngleDeg
    cAngleDeg = change.new
coneAngleSlider.observe(handle_change, 'value')

cameraMode = cameraType.value
def handle_change(change):
    global cameraMode 
    cameraMode = change.new
cameraType.observe(handle_change, 'value')


The Periodic Table. The periodic table below is provided as a sensible way to select the desired atom, just click on it. The many atomic properties the table is known to provide may be taken into consideration while making a choice.

Most of the controls will be displayed in the cell following the periodic table. Note that within those controls, the atom can also be selected by a dropdown widget.

Other control options include: type of proton emissions displayed, color coding of the specific number of protons in each proton stack, proton separation distances, and whether to include an atomic label or not.

In [8]:
grid # Display the Periodic Table

GridspecLayout(children=(Button(description='H', layout=Layout(border='1px solid black', grid_area='widget001'…

In [10]:
guiwidgets # Display the main control widgets. 

In [11]:
# Atom data.

# Operating instructions. Start by clicking on this cell. 
# Next, from the Cell dropdown selection menu choose "Run 
# All Above". When the gui in the preceeding cell above 
# becomes visible, make any changes to it or select an atom 
# from the Periodic table cell. Next, with either the gui cell 
# or the Periodic table cell active, select "Run All Below"
# from the Cell dropdown menu. You'll then view the main 
# program outputs from the tab enclosure. Go back to the gui 
# or Periodic table to make any additional changes, then 
# select "Run All Below".  

# Atom data. After making an element or molecule change, 
# obtain the seleected set of atomic data.

# Selected Atom
An = elements[number-1]
atomicLabel = An[0] 
atomicNumber = number
elementlist = df['Element'].drop_duplicates().array
atom = elementlist[number-1]
atomsymbol = dlist[number-1]
rowstart = 19 * (atomicNumber - 1)
rowstop = rowstart + 18 # Used with the loc slicing
slotsetA_df = df.loc[rowstart:rowstop, 'SlotNumber':'AtomicSlotGrp'].set_index('SlotNumber')
slotlistA = list(slotsetA_df.loc[:, 'Protons'])
                    
#print('slotlistA = ',slotlistA)
#print(slotsetA_df) 
#slotsetA_df.info()  


In [12]:
# Create the Slotlayout Diagram. 

# The slotlist, a set of 19 integers identifying 
# the number of protons in each numbered slot is
# found in the df['Protons'] column.

# Using 45 non-interactive buttons to create the
# atom's slotlayout (19 + 19 + 7 + 1) diagram: 19 
# in the center SL configuration; 19 for the left
# (1,3,5,...,19) and right(2,4,6,...,18) columns;
# 7 for the slot/proton count color legend at the
# bottom; and 1 atomic symbol/mumber at the bottom 
# right. All buttons except the bottom right incl:   
# proton count color, layout width and layout height.

# The 7 colors used for color-coding each slot's 
# total proton counts, 0-6.
sclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     

sdlist = slotlistA

# The colors for the 19 slots, sclist, according to
# the number of protons in each.
sclist = np.zeros(19, dtype='object')
for i in range(19):
    sclist[i] = sclrs[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors
s1 = Button(description='1 Z '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s2 = Button(description='2 Y '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s3 = Button(description='3 Y '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s4 = Button(description='4 Z '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s5 = Button(description='5 Z '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s6 = Button(description='6 Y '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s7 = Button(description='7 Y '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s8 = Button(description='8 X '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s9 = Button(description='9 X '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s10 = Button(description='10 Z '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s11 = Button(description='11 Z '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s12 = Button(description='12 Z '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s13 = Button(description='13 Z '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s14 = Button(description='14 Y '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s15 = Button(description='15 Y '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s16 = Button(description='16 Z '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s17 = Button(description='17 Z '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s18 = Button(description='18 Z '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s19 = Button(description='19 Z '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# Slotlayout proton total color legend 
s20 = Button(description=str('0'), style=dict(button_color=sclrs[0]))
s21 = Button(description=str('1'), style=dict(button_color=sclrs[1]))
s22 = Button(description=str('2'), style=dict(button_color=sclrs[2]))
s23 = Button(description=str('3'), style=dict(button_color=sclrs[3]))
s24 = Button(description=str('4'), style=dict(button_color=sclrs[4]))
s25 = Button(description=str('5'), style=dict(button_color=sclrs[5]))
s26 = Button(description=str('6'), style=dict(button_color=sclrs[6]))
# Left and right slot columns, 
s27 = Button(description='1 Z '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s28 = Button(description='2 Y '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s29 = Button(description='3 Y '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s30 = Button(description='4 Z '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s31 = Button(description='5 Z '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s32 = Button(description='6 Y '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s33 = Button(description='7 Y '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s34 = Button(description='8 X '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s35 = Button(description='9 X '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s36 = Button(description='10 Z '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s37 = Button(description='11 Z '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s38 = Button(description='12 Z '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s39 = Button(description='13 Z '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s40 = Button(description='14 Y '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s41 = Button(description='15 Y '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s42 = Button(description='16 Z '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s43 = Button(description='17 Z '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s44 = Button(description='18 Z '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s45 = Button(description='19 Z '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# The slotlayout atom
s46 = Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0],font_weight='bold'))

# Creating the list of buttons: 19,7,19 and 1.
sbttns = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,
          s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,
          s37,s38,s39,s40,s41,s42,s43,s44,s45,s46]

# Add each button's border, layout width and height.
i=0
while i < 46 :
    sbttns[i].layout = widgets.Layout(border = '1px solid black') 
    sbttns[i].layout.width = '100%'
    sbttns[i].layout.height = '100%' 
    i += 1

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sgrid = GridspecLayout(10, 9)

sgrid[1,4] = sbttns[13] # Cr6-Elements.csv SlotLayout.
sgrid[2,4] = sbttns[3]
sgrid[2,5] = sbttns[8]
sgrid[2,6] = sbttns[12]
sgrid[3,3] = sbttns[17]
sgrid[3,4] = sbttns[1]
sgrid[3,5] = sbttns[15]
sgrid[4,2] = sbttns[9]
sgrid[4,3] = sbttns[5]
sgrid[4,4] = sbttns[0]
sgrid[4,5] = sbttns[6]
sgrid[4,6] = sbttns[11]
sgrid[5,3] = sbttns[16]
sgrid[5,4] = sbttns[2]
sgrid[5,5] = sbttns[18]
sgrid[6,2] = sbttns[10]
sgrid[6,3] = sbttns[7]
sgrid[6,4] = sbttns[4]
sgrid[7,4] = sbttns[14]
sgrid[9,1] = sbttns[19] # Color/num protons legend
sgrid[9,2] = sbttns[20] # along the bottom row
sgrid[9,3] = sbttns[21]
sgrid[9,4] = sbttns[22]
sgrid[9,5] = sbttns[23]
sgrid[9,6] = sbttns[24]
sgrid[9,7] = sbttns[25]
sgrid[0,0] = sbttns[26] # L/R Vertical slot columns
sgrid[0,8] = sbttns[27] 
sgrid[1,0] = sbttns[28]
sgrid[1,8] = sbttns[29]
sgrid[2,0] = sbttns[30]
sgrid[2,8] = sbttns[31]
sgrid[3,0] = sbttns[32]
sgrid[3,8] = sbttns[33]
sgrid[4,0] = sbttns[34]
sgrid[4,8] = sbttns[35]
sgrid[5,0] = sbttns[36]
sgrid[5,8] = sbttns[37]
sgrid[6,0] = sbttns[38]
sgrid[6,8] = sbttns[39]
sgrid[7,0] = sbttns[40]
sgrid[7,8] = sbttns[41]
sgrid[8,0] = sbttns[42]
sgrid[8,8] = sbttns[43]
sgrid[9,0] = sbttns[44]
sgrid[9,8] = sbttns[45] # Atomic symbpl and number


The Slot Layout Diagram.

All atoms can be described by the number of protons (0-6) in each of the atom's (1-19) occupied slot positions.

The main vertical up/down channel slots (top to bottom) are (14,4,2,1,3,5,15). There is also the front/back channel (11,8,1,9,13), and left/right channel (10,6,1,7,12); or refer to the directions from center slot 1: +/-x, +/-y, and main vertical column +/-z. The front/back/ left/right arms spin as a single group, the Carousel, around slot 1. Hydroden has a single proton in its single occupied slot 1. Helium's single slot 1 contains two protons. Hook slot positions: 16,17,18, and 19, are joined to the main up/down column at slots, 2 and 3.

All but most of the bottom row buttons contain number pairs separated by either X, Y or Z, the direction of that slot's proton equatorial emission plane viewed edgewise, orthogonal to the proton's spin axis. The left side number indicates the 1-19 slot number. The right side number indicates the number of protons in that atom's slot.

The Slot Layout is shown in the atomic configuration in the center of the diagram. It is a 3D, six pointed star projected onto the diagram's flat surface. The slot button information is repeated in the left and right side vertical columns for convienence. The seven center buttons in the bottom row contains the total protons per slot color legend. The selected atom’s symbol and atomic number are in the bottom right corner.

In [13]:
""" Create an SL Diagram including slot proton spin directions: 
R, L, A or B; and proton pole electron positions: T, B, or A. An 
'Electrons' value of A means electrons are in a TB configuration, 
at the top proton pole positions at the positive end of slot, and 
at the bottom pole positions at the slot's negative end. BT 
electron positions are not allowed.
A 'SlotSpin' value of A means the 2-directional slot proton at 
the top spins left, while the bottom proton spins right. A value 
of B means the opposite, the proton the top spins right while 
the bottom proton spins left. 
Occupied slots in the center SL contain 5 digits: 1. Electron 
position at the proton's T or B or A (both T and B). 2. The slot 
number. 3. The slot's proton emission plane orientation. 4. The 
color-coded number of protons in the slot. 5. The slot's R,L,A 
(LR) or B (RL) proton spin directions. This atomic model 
simplifies things in that each proton may be accompanied by both 
an electron and a neutron which spin about the proton poles as a 
single object. Each object should spin independently"""

# The 7 colors used for color-coding each slot's total proton counts, 0-6.
snclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     

sndlist = slotlistA

# The colors for the 19 slots, sclist, according to
# the number of protons in each.
snclist = np.zeros(19, dtype='object')
for i in range(19):
    snclist[i] = snclrs[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors.
# If the slot is occupied, include 5 digits, unoccupied slots show 3 digits. 
if slotsetA_df.loc[1,'Protons'] == 0 :
    sn1 = Button(description = '1Z' + str(sndlist[0]), style=dict(button_color=snclist[0]))
elif slotsetA_df.loc[1,'Protons'] > 0 :
    sn1 = Button(description = str(slotsetA_df.loc[1,'Electrons']) + '1Z' + str(sndlist[0]) + str(slotsetA_df.loc[1,'SlotSpin']), style=dict(button_color=snclist[0]))
if slotsetA_df.loc[2,'Protons'] == 0 :
    sn2 = Button(description = '2Y' + str(sndlist[1]), style=dict(button_color=snclist[1]))
elif slotsetA_df.loc[2,'Protons'] > 0 :
    sn2 = Button(description = str(slotsetA_df.loc[2,'Electrons']) + '2Y' + str(sndlist[1]) + str(slotsetA_df.loc[2,'SlotSpin']), style=dict(button_color=snclist[1]))
if slotsetA_df.loc[3,'Protons'] == 0 :
    sn3 = Button(description = '3Y' + str(sndlist[2]), style=dict(button_color=snclist[2]))
elif slotsetA_df.loc[3,'Protons'] > 0 :
    sn3 = Button(description = str(slotsetA_df.loc[3,'Electrons']) + '3Y' + str(sndlist[2]) + str(slotsetA_df.loc[3,'SlotSpin']), style=dict(button_color=snclist[2]))
if slotsetA_df.loc[4,'Protons'] == 0 :
    sn4 = Button(description = '4Z' + str(sndlist[3]), style=dict(button_color=snclist[3]))
elif slotsetA_df.loc[4,'Protons'] > 0 :
    sn4 = Button(description = str(slotsetA_df.loc[4,'Electrons']) + '4Z' + str(sndlist[3]) + str(slotsetA_df.loc[4,'SlotSpin']), style=dict(button_color=snclist[3]))
if slotsetA_df.loc[5,'Protons'] == 0 :
    sn5 = Button(description = '5Z' + str(sndlist[4]), style=dict(button_color=snclist[4]))
elif slotsetA_df.loc[5,'Protons'] > 0 :
    sn5 = Button(description = str(slotsetA_df.loc[5,'Electrons']) + '5Z' + str(sndlist[4]) + str(slotsetA_df.loc[5,'SlotSpin']), style=dict(button_color=snclist[4]))
if slotsetA_df.loc[6,'Protons'] == 0 :
    sn6 = Button(description = '6Y' + str(sndlist[5]), style=dict(button_color=snclist[5]))
elif slotsetA_df.loc[6,'Protons'] > 0 :
    sn6 = Button(description = str(slotsetA_df.loc[6,'Electrons']) + '6Y' + str(sndlist[5]) + str(slotsetA_df.loc[6,'SlotSpin']), style=dict(button_color=snclist[5]))
if slotsetA_df.loc[7,'Protons'] == 0 :
    sn7 = Button(description = '7Y' + str(sndlist[6]), style=dict(button_color=snclist[6]))
elif slotsetA_df.loc[7,'Protons'] > 0 :
    sn7 = Button(description = str(slotsetA_df.loc[7,'Electrons']) + '7Y' + str(sndlist[6]) + str(slotsetA_df.loc[7,'SlotSpin']), style=dict(button_color=snclist[6]))
if slotsetA_df.loc[8,'Protons'] == 0 :
    sn8 = Button(description = '8X' + str(sndlist[7]), style=dict(button_color=snclist[7]))
elif slotsetA_df.loc[8,'Protons'] > 0 :
    sn8 = Button(description = str(slotsetA_df.loc[8,'Electrons']) + '8X' + str(sndlist[7]) + str(slotsetA_df.loc[8,'SlotSpin']), style=dict(button_color=snclist[7]))
if slotsetA_df.loc[9,'Protons'] == 0 :
    sn9 = Button(description = '9X' + str(sndlist[8]), style=dict(button_color=snclist[8]))
elif slotsetA_df.loc[9,'Protons'] > 0 :
    sn9 = Button(description = str(slotsetA_df.loc[9,'Electrons']) + '9X' + str(sndlist[8]) + str(slotsetA_df.loc[9,'SlotSpin']), style=dict(button_color=snclist[8]))
if slotsetA_df.loc[10,'Protons'] == 0 :
    sn10 = Button(description = 'AZ' + str(sndlist[9]), style=dict(button_color=snclist[9]))
elif slotsetA_df.loc[10,'Protons'] > 0 :
    sn10 = Button(description = str(slotsetA_df.loc[10,'Electrons']) + 'AZ' + str(sndlist[9]) + str(slotsetA_df.loc[10,'SlotSpin']), style=dict(button_color=snclist[9]))
if slotsetA_df.loc[11,'Protons'] == 0 :
    sn11 = Button(description = 'BZ' + str(sndlist[10]), style=dict(button_color=snclist[10]))
elif slotsetA_df.loc[11,'Protons'] > 0 :
    sn11 = Button(description = str(slotsetA_df.loc[11,'Electrons']) + 'BZ' + str(sndlist[10]) + str(slotsetA_df.loc[11,'SlotSpin']), style=dict(button_color=snclist[10]))
if slotsetA_df.loc[12,'Protons'] == 0 :
    sn12 = Button(description = 'CZ' + str(sndlist[11]), style=dict(button_color=snclist[11]))
elif slotsetA_df.loc[12,'Protons'] > 0 :
    sn12 = Button(description = str(slotsetA_df.loc[12,'Electrons']) + 'CZ' + str(sndlist[11]) + str(slotsetA_df.loc[12,'SlotSpin']), style=dict(button_color=snclist[11]))
if slotsetA_df.loc[13,'Protons'] == 0 :
    sn13 = Button(description = 'DZ' + str(sndlist[12]), style=dict(button_color=snclist[12]))
elif slotsetA_df.loc[13,'Protons'] > 0 :
    sn13 = Button(description = str(slotsetA_df.loc[13,'Electrons']) + 'DZ' + str(sndlist[12]) + str(slotsetA_df.loc[13,'SlotSpin']), style=dict(button_color=snclist[12]))
if slotsetA_df.loc[14,'Protons'] == 0 :
    sn14 = Button(description = 'EY' + str(sndlist[13]), style=dict(button_color=snclist[13]))
elif slotsetA_df.loc[14,'Protons'] > 0 :
    sn14 = Button(description = str(slotsetA_df.loc[14,'Electrons']) + 'EY' + str(sndlist[13]) + str(slotsetA_df.loc[14,'SlotSpin']), style=dict(button_color=snclist[13]))
if slotsetA_df.loc[15,'Protons'] == 0 :
    sn15 = Button(description = 'FY' + str(sndlist[14]), style=dict(button_color=snclist[14]))
elif slotsetA_df.loc[15,'Protons'] > 0 :
    sn15 = Button(description = str(slotsetA_df.loc[15,'Electrons']) + 'FY' + str(sndlist[14]) + str(slotsetA_df.loc[15,'SlotSpin']), style=dict(button_color=snclist[14]))
if slotsetA_df.loc[16,'Protons'] == 0 :
    sn16 = Button(description = 'GZ' + str(sndlist[15]), style=dict(button_color=snclist[15]))
elif slotsetA_df.loc[16,'Protons'] > 0 :
    sn16 = Button(description = str(slotsetA_df.loc[16,'Electrons']) + 'GZ' + str(sndlist[15]) + str(slotsetA_df.loc[16,'SlotSpin']), style=dict(button_color=snclist[15]))
if slotsetA_df.loc[17,'Protons'] == 0 :
    sn17 = Button(description = 'HZ' + str(sndlist[16]), style=dict(button_color=snclist[16]))
elif slotsetA_df.loc[17,'Protons'] > 0 :
    sn17 = Button(description = str(slotsetA_df.loc[17,'Electrons']) + 'HZ' + str(sndlist[16]) + str(slotsetA_df.loc[17,'SlotSpin']), style=dict(button_color=snclist[16]))
if slotsetA_df.loc[18,'Protons'] == 0 :
    sn18 = Button(description = 'IZ' + str(sndlist[17]), style=dict(button_color=snclist[17]))
elif slotsetA_df.loc[18,'Protons'] > 0 :
    sn18 = Button(description = str(slotsetA_df.loc[18,'Electrons']) + 'IZ' + str(sndlist[17]) + str(slotsetA_df.loc[18,'SlotSpin']), style=dict(button_color=snclist[17]))
if slotsetA_df.loc[19,'Protons'] == 0 :
    sn19 = Button(description = 'JZ' + str(sndlist[18]), style=dict(button_color=snclist[18]))
elif slotsetA_df.loc[19,'Protons'] > 0 :
    sn19 = Button(description = str(slotsetA_df.loc[19,'Electrons']) + 'JZ' + str(sndlist[18]) + str(slotsetA_df.loc[19,'SlotSpin']), style=dict(button_color=snclist[18]))

# Slotlayout proton total color legend 
sn20 = Button(description=str('0'), style=dict(button_color=snclrs[0]))
sn21 = Button(description=str('1'), style=dict(button_color=snclrs[1]))
sn22 = Button(description=str('2'), style=dict(button_color=snclrs[2]))
sn23 = Button(description=str('3'), style=dict(button_color=snclrs[3]))
sn24 = Button(description=str('4'), style=dict(button_color=snclrs[4]))
sn25 = Button(description=str('5'), style=dict(button_color=snclrs[5]))
sn26 = Button(description=str('6'), style=dict(button_color=snclrs[6]))
# Left and right slot columns, 
#sn27 = Button(description='1 Z '+str(sndlist[0]), style=dict(button_color=snclist[0]))
sn27 = Button(description = '1Z' + str(sndlist[0]), style=dict(button_color=snclist[0]))
sn28 = Button(description = '2Y' + str(sndlist[1]), style=dict(button_color=snclist[1]))
sn29 = Button(description = '3Y' + str(sndlist[2]), style=dict(button_color=snclist[2]))
sn30 = Button(description = '4Z' + str(sndlist[3]), style=dict(button_color=snclist[3]))
sn31 = Button(description = '5Z' + str(sndlist[4]), style=dict(button_color=snclist[4]))
sn32 = Button(description = '6Y' + str(sndlist[5]), style=dict(button_color=snclist[5]))
sn33 = Button(description = '7Y' + str(sndlist[6]), style=dict(button_color=snclist[6]))
sn34 = Button(description = '8X' + str(sndlist[7]), style=dict(button_color=snclist[7]))
sn35 = Button(description = '9X' + str(sndlist[8]), style=dict(button_color=snclist[8]))
sn36 = Button(description = 'AZ' + str(sndlist[9]), style=dict(button_color=snclist[9]))
sn37 = Button(description = 'BZ' + str(sndlist[10]), style=dict(button_color=snclist[10]))
sn38 = Button(description = 'CZ' + str(sndlist[11]), style=dict(button_color=snclist[11]))
sn39 = Button(description = 'DZ' + str(sndlist[12]), style=dict(button_color=snclist[12]))
sn40 = Button(description = 'EY' + str(sndlist[13]), style=dict(button_color=snclist[13]))
sn41 = Button(description = 'FY' + str(sndlist[14]), style=dict(button_color=snclist[14]))
sn42 = Button(description = 'GZ' + str(sndlist[15]), style=dict(button_color=snclist[15]))
sn43 = Button(description = 'HZ' + str(sndlist[16]), style=dict(button_color=snclist[16]))
sn44 = Button(description = 'IZ' + str(sndlist[17]), style=dict(button_color=snclist[17]))
sn45 = Button(description = 'JZ' + str(sndlist[18]), style=dict(button_color=snclist[18]))
# The slotlayout atom
sn46 = Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0],font_weight='bold'))

# Creating the list of buttons: 19,7,19 and 1.
snbttns = [sn1,sn2,sn3,sn4,sn5,sn6,sn7,sn8,sn9,sn10,sn11,sn12,sn13,sn14,sn15,sn16,sn17,sn18,sn19,
          sn20,sn21,sn22,sn23,sn24,sn25,sn26,sn27,sn28,sn29,sn30,sn31,sn32,sn33,sn34,sn35,sn36,
          sn37,sn38,sn39,sn40,sn41,sn42,sn43,sn44,sn45,sn46]

# Add each button's border, layout width and height.
i=0
while i < 46 :
    snbttns[i].layout = widgets.Layout(border = '1px solid black') 
    snbttns[i].layout.width = '100%'
    snbttns[i].layout.height = '100%' 
    i += 1

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sngrid = GridspecLayout(10, 9)

sngrid[1,4] = snbttns[13] # Cr6-Elements.csv SlotLayout.
sngrid[2,4] = snbttns[3]
sngrid[2,5] = snbttns[8]
sngrid[2,6] = snbttns[12]
sngrid[3,3] = snbttns[17]
sngrid[3,4] = snbttns[1]
sngrid[3,5] = snbttns[15]
sngrid[4,2] = snbttns[9]
sngrid[4,3] = snbttns[5]
sngrid[4,4] = snbttns[0]
sngrid[4,5] = snbttns[6]
sngrid[4,6] = snbttns[11]
sngrid[5,3] = snbttns[16]
sngrid[5,4] = snbttns[2]
sngrid[5,5] = snbttns[18]
sngrid[6,2] = snbttns[10]
sngrid[6,3] = snbttns[7]
sngrid[6,4] = snbttns[4]
sngrid[7,4] = snbttns[14]
sngrid[9,1] = snbttns[19] # Color/num protons legend
sngrid[9,2] = snbttns[20] # along the bottom row
sngrid[9,3] = snbttns[21]
sngrid[9,4] = snbttns[22]
sngrid[9,5] = snbttns[23]
sngrid[9,6] = snbttns[24]
sngrid[9,7] = snbttns[25]
sngrid[0,0] = snbttns[26] # L/R Vertical slot columns
sngrid[0,8] = snbttns[27] 
sngrid[1,0] = snbttns[28]
sngrid[1,8] = snbttns[29]
sngrid[2,0] = snbttns[30]
sngrid[2,8] = snbttns[31]
sngrid[3,0] = snbttns[32]
sngrid[3,8] = snbttns[33]
sngrid[4,0] = snbttns[34]
sngrid[4,8] = snbttns[35]
sngrid[5,0] = snbttns[36]
sngrid[5,8] = snbttns[37]
sngrid[6,0] = snbttns[38]
sngrid[6,8] = snbttns[39]
sngrid[7,0] = snbttns[40]
sngrid[7,8] = snbttns[41]
sngrid[8,0] = snbttns[42]
sngrid[8,8] = snbttns[43]
sngrid[9,0] = snbttns[44]
sngrid[9,8] = snbttns[45] # Atomic symbol and number


In [14]:
# Build each atom from Cr6-Elements

# atomicNumber = number  
# Select the desired atom from the dropdown widget or table.
if 1 <= number < 13:
    caroSecs = 2.5
if 14 <= number < 20:
    caroSecs = 4.0
if 21 <= number :
    caroSecs = 6.0

grpA1 = Group()  # Carousel. The corner rotation widget spins grpA1, 
# slots 6,7,8,9,10,11,12,13.
grpA2 = Group()  # grpA2 contains slot1 and the remaining non-carousel
# orthogonal top, bottom and hook slots.
grpB1 = Group()  # Atom 2 Slot 1 and carousel
grpB2 = Group()  # Atom 2 remaining slots

# Each of the 1-19 slots of a given atom may have either right 
# (A stacks) or left (B stacks) spinning proton groups. 38 Total.
proStackA1 = Group()  
proStackA2 = Group()  
proStackA3 = Group()
proStackA4 = Group()
proStackA5 = Group()
proStackA6 = Group()
proStackA7 = Group()
proStackA8 = Group()
proStackA9 = Group()
proStackA10 = Group()
proStackA11 = Group()
proStackA12 = Group()
proStackA13 = Group()
proStackA14 = Group()
proStackA15 = Group()
proStackA16 = Group()
proStackA17 = Group()
proStackA18 = Group()
proStackA19 = Group()

stacksA = [proStackA1,proStackA2,proStackA3,proStackA4,proStackA5,
          proStackA6,proStackA7,proStackA8,proStackA9,proStackA10,
          proStackA11,proStackA12,proStackA13,proStackA14,proStackA15,
          proStackA16,proStackA17,proStackA18,proStackA19]

proStackB1 = Group()  
proStackB2 = Group()  
proStackB3 = Group()
proStackB4 = Group()
proStackB5 = Group()
proStackB6 = Group()
proStackB7 = Group()
proStackB8 = Group()
proStackB9 = Group()
proStackB10 = Group()
proStackB11 = Group()
proStackB12 = Group()
proStackB13 = Group()
proStackB14 = Group()
proStackB15 = Group()
proStackB16 = Group()
proStackB17 = Group()
proStackB18 = Group()
proStackB19 = Group()

stacksB = [proStackB1,proStackB2,proStackB3,proStackB4,proStackB5,
          proStackB6,proStackB7,proStackB8,proStackB9,proStackB10,
          proStackB11,proStackB12,proStackB13,proStackB14,proStackB15,
          proStackB16,proStackB17,proStackB18,proStackB19]

# Dimensions # pRadius is now in cell In[1]
nRadius = pRadius*1.125 # Neutron radius
#eRadius = 2*pRadius/10 # Electron radius
eRadius = 2*pRadius/5 # Electron radius

""" 
Widget Controlled variables 1. ortho, proton1 equator to 
proton2 pole. 2.emR, the equatorial(or cone) emission radius.
3. cAngleDeg, the cone angle. 4. pPTP, dist between an alpha's
protons. 5.p2p3Dist, between 2 adjacent alphas 
"""
pE = ortho*pRadius
emRadius = 2*emR*pRadius # 2*pE - pRadius/2
coneAngle = cAngleDeg*2*piG/360
coneHeight = emRadius*tan(coneAngle)
p12 = 2*pPTP*pRadius 
p2p3 = 2*p2p3Dist*pRadius
n1n2 = (p12- 2*nRadius)/tan(coneAngle) # neutron-neutron orbital separation (y) inside the alpha
# e1e2 = -n1n2/10
p1p2 = pRadius*2.5  #0.75*p12 # pRadius*2.5
p1n1 = p12/2 #p12/2 #p12/2 # Alpha, up-down (z) proton-orbital neutron separation
p1n2 = p12/2 #pRadius + nRadius + 1.5*pRadius  # In line proton and neutron
p1e1 = pRadius + 0.05*p12 # Alpha, top-bottom (z) proton-electron separation

# Proton set Geometries and Materials # Color.js Find a nice color spectrum
protonGeometry =  SphereBufferGeometry(pRadius, 24, 16)
neutronGeometry =  SphereBufferGeometry(nRadius, 24, 16)
electronGeometry =  SphereBufferGeometry(eRadius, 8, 16)
emissionGeometryP = CircleGeometry(emRadius, 24)
emissionGeometryX = ConeGeometry( emRadius, coneHeight, 16, openEnded = True )
protonMaterial = MeshStandardMaterial(color='blue')
neutronMaterial = MeshStandardMaterial(color='red')
electronMaterial = MeshStandardMaterial(color='cyan') #'orangered','salmon','green','cyan','teal','orange'
emissionmaterial7= MeshStandardMaterial(color='gray',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial1 = MeshStandardMaterial(color='magenta',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial2 = MeshStandardMaterial(color='blue',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial3 = MeshStandardMaterial(color='pink',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial4 = MeshStandardMaterial(color='salmon',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial5 = MeshStandardMaterial(color='lime',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial6 = MeshStandardMaterial(color='skyblue',side='DoubleSide',transparent = True,opacity = 0.20)

emissionMaterialList = [emissionmaterial1,emissionmaterial2,emissionmaterial3,
        emissionmaterial4,emissionmaterial5,emissionmaterial6,emissionmaterial7]

if emissionType.value == 'plane':
    emissionGeometry = emissionGeometryP
if emissionType.value == 'cones':
    emissionGeometry = emissionGeometryX

def buildProStackM(a,j): 
    """
    Parameters: a - total number of slot protons. j - given slot #.
    
    buildProStackM(a,j) is an attempt to follow Miles Mathis' 
    paper,'Diatomic Hydrogen' (DH), which includes electron-proton 
    (e-p) to e-p charge bonding summarized in rules 1, 2, and 3: 
    Rule 1. Two protons, each with any spin direction (L or R) can 
    bond when they share a T B electron position configuration.
    Rule 2. T T or B B electron position configurations will be valid 
    only if both protons have the same (L or R) spin direction.
    Rule 3. No combination of B T electrons are valid, two electrons 
    between two protons prevent bonding.
    """
    
    if coloredEmissions.value == True:
        for k in range(6): 
            if a == k + 1 :
                emissionmaterial = emissionMaterialList[k]
    else:
        emissionmaterial = emissionMaterialList[6]              

    """ 
    p2p3 = 2*p2p3Dist*pRadius, distance added for each additional e-p.
    The six sets of equidistant proton positions for a slot with 1-6 protons. 
    The proton (groups) are added beginning at the slot center. 5,3,1,2,4,6.
    """
    if a == 1 : pPositions = [0]
    if a == 2 : pPositions = [p2p3/2,-p2p3/2]
    if a == 3 : pPositions = [0,-p2p3, p2p3]
    if a == 4 : pPositions = [p2p3/2,-p2p3/2,3*p2p3/2,-3*p2p3/2]
    if a == 5 : pPositions = [0,-p2p3,p2p3,-2*p2p3,2*p2p3]        
    if a == 6 : pPositions = [p2p3/2,-p2p3/2,3*p2p3/2,-3*p2p3/2,5*p2p3/2,-5*p2p3/2]  
    
    def conesPlusPlane(protonMi,emission1,emission2,position):
        emission1.add(Mesh(emissionGeometry, emissionmaterial))
        #emission1.position = [0,0,coneHeight/2]
        emission1.position = [0,0,position]
        emission1.rotateX(-piG/2)
        protonMi.add(emission1)
        emission2.add(Mesh(emissionGeometry, emissionmaterial))
        #emission2.position = [0,0,-coneHeight/2]
        emission2.position = [0,0,-position]
        emission2.rotateX(piG/2)  
        protonMi.add(emission2)            
        protonMi.add(Mesh(emissionGeometryP, emissionmaterial))       
        return

    def addElectron(protonMi,electronMi,position):
        electronMi.add(Mesh(electronGeometry, electronMaterial))
        electronMi.position = [0,-n1n2/10,position]
        electronMi.rotateZ(random()*2*piG)
        protonMi.add(electronMi)    
        return
    
    def singleNeutron(protonMi,neutron,position): 
        neutron.add(Mesh(neutronGeometry, neutronMaterial))
        neutron.position = [0,-0.75*n1n2/2,position]  
        neutron.rotateZ(random()*2*piG)
        if displayNeutrons is True :
            protonMi.add(neutron) 
        return  

    def pairOfNeutrons(protonMi,neutron1,neutron2,position):
        neutron1.add(Mesh(neutronGeometry, neutronMaterial))
        neutron1.position = [0,-0.75*n1n2/2,0]
        neutron2.add(Mesh(neutronGeometry, neutronMaterial))
        neutron2.position = [0,0.75*n1n2/2,0]
        neutronPair.add(neutron1)
        neutronPair.add(neutron2)
        neutronPair.rotateZ(random()*2*piG)
        neutronPair.position = [0,0,position]
        if displayNeutrons is True :
            protonMi.add(neutronPair)        
        return

    num = 0 # Increments the occupied slot's 1-6 proton groups.  
    while num < a:
        protonMi = Group()   
        protonMi.add(Mesh(protonGeometry, protonMaterial))
        if emissionType.value == 'plane':
            protonMi.add(Mesh(emissionGeometry, emissionmaterial))
        if emissionType.value == 'cones': 
            emission1 = Object3D() 
            emission2 = Object3D()
            conesPlusPlane(protonMi,emission1,emission2,coneHeight/2)

        # Electron and neutron additions to proton spin groups. 
        electronMi = Object3D()        
        neutronMi = Object3D()
        neutronNi = Object3D()
        neutronPair = Group()

        # Building the various T,B,A, and R,L,A slot configurations. 
        intColumn = 8+num  # Pointing to the N1-N6 data locations. 
        intValue = slotsetA_df.iloc[j,intColumn]
        if slotsetA_df.loc[j+1,('SlotSpin')] == "R" or "L" : 
            if slotsetA_df.loc[j+1,('Electrons')] == "T" : 
                addElectron(protonMi,electronMi,p1e1)             
                if intValue == 1 :                  
                    singleNeutron(protonMi,neutronMi,-p2p3/2)               
                elif intValue == 2 :  
                    pairOfNeutrons(protonMi,neutronMi,neutronNi,-p2p3/2)
            if slotsetA_df.loc[j+1,('Electrons')] == "B" : 
                addElectron(protonMi,electronMi,-p1e1)
                if intValue == 1 :  
                    singleNeutron(protonMi,neutronMi,p2p3/2)
                elif intValue == 2 :
                    pairOfNeutrons(protonMi,neutronMi,neutronNi,p2p3/2)
            if slotsetA_df.loc[j+1,('Electrons')] == "A" : # 'SlotSpin' = R or L
                if num%2 != 0:
                    addElectron(protonMi,electronMi,-p1e1)
                    if intValue == 1 :                     
                        singleNeutron(protonMi,neutronMi,p2p3/2)                    
                    elif intValue == 2 :
                        pairOfNeutrons(protonMi,neutronMi,neutronNi,p2p3/2)                    
                elif num%2 == 0:
                    addElectron(protonMi,electronMi,p1e1)
                    if intValue == 1 :                     
                        singleNeutron(protonMi,neutronMi,-p2p3/2)                    
                    elif intValue == 2 :
                        pairOfNeutrons(protonMi,neutronMi,neutronNi,-p2p3/2)  
        if slotsetA_df.loc[j+1,('SlotSpin')] == "A" and slotsetA_df.loc[j+1,('Electrons')] == "A":
            if num == 0 :
                addElectron(protonMi,electronMi,p1e1)
                if intValue == 1 :                  
                    singleNeutron(protonMi,neutronMi,p2p3/2)                
            if num == 1 :
                if intValue == 0 :   
                    addElectron(protonMi,electronMi,-p1e1)
                if intValue == 1 :
                    addElectron(protonMi,electronMi,p1e1)
                    singleNeutron(protonMi,neutronMi,-p2p3/2)   
                if intValue == 2 :
                    addElectron(protonMi,electronMi,p1e1)
                    pairOfNeutrons(protonMi,neutronMi,neutronNi,-p2p3/2)                  
                    protonMi.rotateX(piG)
                    protonMi.rotateZ(random()*2*piG)
            if num > 1 and num%2 == 0:
                addElectron(protonMi,electronMi,p1e1)                
                if intValue == 1 :                     
                    singleNeutron(protonMi,neutronMi,-p2p3/2)
                elif intValue == 2 :
                    pairOfNeutrons(protonMi,neutronMi,neutronNi,-p2p3/2)
            if num > 1 and num%2 != 0:          
                if intValue == 1 :                     
                    addElectron(protonMi,electronMi,-p1e1)
                    singleNeutron(protonMi,neutronMi,p2p3/2)
                elif intValue == 2 :
                    addElectron(protonMi,electronMi,-p1e1)
                    pairOfNeutrons(protonMi,neutronMi,neutronNi,p2p3/2)
      
        protonMi.rotateZ(random()*2*piG)
        protonMi.position = [0,0,pPositions[num]]
        
        # Slot spin assignments
        if slotsetA_df.loc[j+1,('SlotSpin')] == "R":                 
            stacksA[j].add(protonMi)
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "L" :
            stacksB[j].add(protonMi)
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "A" :        
            # Double spin slot spin assignments.
            if num%2 != 0:            
                stacksA[j].add(protonMi)
            else:
                stacksB[j].add(protonMi)  
        elif slotsetA_df.loc[j+1,('SlotSpin')] == "B" :        
            # Double spin slot spin assignments.
            if num%2 != 0:            
                stacksB[j].add(protonMi)
            else:
                stacksA[j].add(protonMi)               
               
        num += 1
    return

def Orient(n, stack):
    # proStack emission planes are initially oriented in the z-plane.
    # Reorient the stack otherwise according to the "SlotOrien" column.
    if slotsetA_df.loc[n+1,('SlotOrien')] == "Y" : 
        stack.rotateX(-piG/2)            
    if slotsetA_df.loc[n+1,('SlotOrien')] == "X" :
        stack.rotateX(-piG/2)
        stack.rotateY(piG/2)
    return            

def Position(n,stack):
    # The default values are just used as position markers, +/- x, y or z 
    # with respect to (0,0,0). Use those directions but calculate the slot's 
    # position magnitude based on the values of p2p3, p12 and pE. 
    loc = (abs(slotsetA_df.loc[n+1,('P2P3')]*p2p3 
               + slotsetA_df.loc[n+1,('P12')]*p12 
               + slotsetA_df.loc[n+1,('PE')]*pE))
    # Determine the direction of the stack according to the presence and sign  
    # (+/-) of slotsetA_df's occupied (x,y,z) positions. Either a value or a
    #  zero, '0'. If there's a value, let it be given the position magnitude.
    px = slotsetA_df.loc[i+1,('PX')] #
    py = slotsetA_df.loc[i+1,('PY')] 
    pz = slotsetA_df.loc[i+1,('PZ')] 
    if px != 0 : px = copysign(1, px) 
    if py != 0 : py = copysign(1, py) 
    if pz != 0 : pz = copysign(1, pz)             
    stack.position = [px*loc, py*loc, pz*loc]
    return

# Assemble the atomic models.
for i in range( 19 ):
    if slotsetA_df.loc[i+1,('Protons')] > 0 :   
        numPs = slotsetA_df.loc[i+1,('Protons')]
        buildProStackM(numPs,i)     
        Orient(i,stacksA[i])
        Orient(i,stacksB[i])
        Position(i,stacksA[i])
        Position(i,stacksB[i])
        #grpA1: Carousal slots: 6,7,8,9,10,11,12,13
        anSlots=[5,6,7,8,9,10,11,12] # index#=slot-1        
        if i in anSlots:       
            grpA1.add(stacksA[i])
            grpA1.add(stacksB[i])
        else:
            grpA2.add(stacksA[i]) 
            grpA2.add(stacksB[i])   

print(An) # (' 2 helium He ', 2)
print(slotlistA) 
#print('moleculeOrNot ,',moleculeOrNot.value,', di-atom ,',numberM) 
#slotsetA_df.to_csv('out.csv') 

#slotsetA_df  

(' 45 rhodium Rh ', 45)
[4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [15]:
# The rendered scene

if atomsLabel.value == True:   # Still shows UserWarning
    scene = Scene(children=[grpA1, grpA2, grpB1, grpB2, camera, key_light, ambient_light,
        make_text(atomicLabel,
                  height=0.25,
                  position = [-1,1,number*0.025 + 1.5])])
else:
    scene = Scene(children=[grpA1, grpA2, grpB1, grpB2, camera, key_light, ambient_light])

controller = OrbitControls(controlling=camera, screenSpacePanning=True)
camera.mode = cameraType.value
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)],
                    width=view_width, height=view_height)


# initialising the scene
scene = Scene()

# Note. The UserWarning appears to be due to non JSON compliant infinite 
# values used in the pythreejs orbital camera. See the related issue. JSON 
# serialization of OrbitControls fails with jupyter_client 7.0.3,opened 
# on 22 Sep 21, https://github.com/jupyter-widgets/pythreejs/issues/366

# 16 April, switched to a combined camera, allowing the user to switch views 
# https://github.com/jupyter-widgets/pythreejs/blob/master/examples/Combined%20Camera.ipynb
# Still getting a User Warning. 

C:\Users\Robert\anaconda3\envs\New_environ\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [16]:
# Atom 1 spin animations. Defining the rotation widget controls for slots 1 - 19. 
# Slot 1 (and the carousel).
aTrack = NumberKeyframeTrack(name='.rotation[z]', times=[0, caroSecs], values=[0, 2*piG])
# Slots 2-19.
bTrack = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 2*piG])

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19. 
spinA1_track = bTrack
spinA1_clip = AnimationClip(tracks=[spinA1_track])
spinA1_action = AnimationAction(AnimationMixer(proStackA1), spinA1_clip, proStackA1)

# The remaining widgets rotate the individual slots. 
spinA2_track = bTrack
spinA2_clip = AnimationClip(tracks=[spinA2_track])
spinA2_action = AnimationAction(AnimationMixer(proStackA2), spinA2_clip, proStackA2)

spinA3_track = bTrack
spinA3_clip = AnimationClip(tracks=[spinA3_track])
spinA3_action = AnimationAction(AnimationMixer(proStackA3), spinA3_clip, proStackA3)

spinA4_track = bTrack
spinA4_clip = AnimationClip(tracks=[spinA4_track])
spinA4_action = AnimationAction(AnimationMixer(proStackA4), spinA4_clip, proStackA4)

spinA5_track = bTrack
spinA5_clip = AnimationClip(tracks=[spinA5_track])
spinA5_action = AnimationAction(AnimationMixer(proStackA5), spinA5_clip, proStackA5)

spinA6_track = bTrack
spinA6_clip = AnimationClip(tracks=[spinA6_track])
spinA6_action = AnimationAction(AnimationMixer(proStackA6), spinA6_clip, proStackA6)

spinA7_track = bTrack
spinA7_clip = AnimationClip(tracks=[spinA7_track])
spinA7_action = AnimationAction(AnimationMixer(proStackA7), spinA7_clip, proStackA7)

spinA8_track = bTrack
spinA8_clip = AnimationClip(tracks=[spinA8_track])
spinA8_action = AnimationAction(AnimationMixer(proStackA8), spinA8_clip, proStackA8)

spinA9_track = bTrack
spinA9_clip = AnimationClip(tracks=[spinA9_track])
spinA9_action = AnimationAction(AnimationMixer(proStackA9), spinA9_clip, proStackA9)

spinA10_track = bTrack
spinA10_clip = AnimationClip(tracks=[spinA10_track])
spinA10_action = AnimationAction(AnimationMixer(proStackA10), spinA10_clip, proStackA10)

spinA11_track = bTrack
spinA11_clip = AnimationClip(tracks=[spinA11_track])
spinA11_action = AnimationAction(AnimationMixer(proStackA11), spinA11_clip, proStackA11)

spinA12_track = bTrack
spinA12_clip = AnimationClip(tracks=[spinA12_track])
spinA12_action = AnimationAction(AnimationMixer(proStackA12), spinA12_clip, proStackA12)

spinA13_track = bTrack
spinA13_clip = AnimationClip(tracks=[spinA13_track])
spinA13_action = AnimationAction(AnimationMixer(proStackA13), spinA13_clip, proStackA13)

spinA14_track = bTrack
spinA14_clip = AnimationClip(tracks=[spinA14_track])
spinA14_action = AnimationAction(AnimationMixer(proStackA14), spinA14_clip, proStackA14)

spinA15_track = bTrack
spinA15_clip = AnimationClip(tracks=[spinA15_track])
spinA15_action = AnimationAction(AnimationMixer(proStackA15), spinA15_clip, proStackA15)

spinA16_track = bTrack
spinA16_clip = AnimationClip(tracks=[spinA16_track])
spinA16_action = AnimationAction(AnimationMixer(proStackA16), spinA16_clip, proStackA16)

spinA17_track = bTrack
spinA17_clip = AnimationClip(tracks=[spinA17_track])
spinA17_action = AnimationAction(AnimationMixer(proStackA17), spinA17_clip, proStackA17)

spinA18_track = bTrack
spinA18_clip = AnimationClip(tracks=[spinA18_track])
spinA18_action = AnimationAction(AnimationMixer(proStackA18), spinA18_clip, proStackA18)

spinA19_track = bTrack
spinA19_clip = AnimationClip(tracks=[spinA19_track])
spinA19_action = AnimationAction(AnimationMixer(proStackA19), spinA19_clip, proStackA19)

spinA20_track = aTrack
spinA20_clip = AnimationClip(tracks=[spinA20_track])
spinA20_action = AnimationAction(AnimationMixer(grpA1), spinA20_clip, grpA1)

rgridA = GridspecLayout(8, 7)

rgridA[1, 3] = spinA14_action
rgridA[2, 3] = spinA4_action
rgridA[2, 4] = spinA9_action
rgridA[2, 5] = spinA13_action
rgridA[3, 2] = spinA18_action
rgridA[3, 3] = spinA2_action
rgridA[3, 4] = spinA16_action
rgridA[4, 1] = spinA10_action
rgridA[4, 2] = spinA6_action
rgridA[4, 3] = spinA1_action
rgridA[4, 4] = spinA7_action
rgridA[4, 5] = spinA12_action
rgridA[5, 2] = spinA17_action
rgridA[5, 3] = spinA3_action
rgridA[5, 4] = spinA19_action
rgridA[6, 1] = spinA11_action
rgridA[6, 2] = spinA8_action
rgridA[6, 3] = spinA5_action
rgridA[7, 3] = spinA15_action
rgridA[7, 5] = spinA20_action

In [17]:
# Second set of spin animations. Defining the rotation widget controls for slots 1 - 19. 
# Slot 1 (and the carousel).
aTrack = NumberKeyframeTrack(name='.rotation[z]', times=[0, caroSecs], values=[0, -2*piG])
# Slots 2-19.
bTrack = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, -2*piG])

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19. 
spinB1_track = bTrack
spinB1_clip = AnimationClip(tracks=[spinB1_track])
spinB1_action = AnimationAction(AnimationMixer(proStackB1), spinB1_clip, proStackB1)

# The remaining widgets rotate the individual slots. 
spinB2_track = bTrack
spinB2_clip = AnimationClip(tracks=[spinB2_track])
spinB2_action = AnimationAction(AnimationMixer(proStackB2), spinB2_clip, proStackB2)

spinB3_track = bTrack
spinB3_clip = AnimationClip(tracks=[spinB3_track])
spinB3_action = AnimationAction(AnimationMixer(proStackB3), spinB3_clip, proStackB3)

spinB4_track = bTrack
spinB4_clip = AnimationClip(tracks=[spinB4_track])
spinB4_action = AnimationAction(AnimationMixer(proStackB4), spinB4_clip, proStackB4)

spinB5_track = bTrack
spinB5_clip = AnimationClip(tracks=[spinB5_track])
spinB5_action = AnimationAction(AnimationMixer(proStackB5), spinB5_clip, proStackB5)

spinB6_track = bTrack
spinB6_clip = AnimationClip(tracks=[spinB6_track])
spinB6_action = AnimationAction(AnimationMixer(proStackB6), spinB6_clip, proStackB6)

spinB7_track = bTrack
spinB7_clip = AnimationClip(tracks=[spinB7_track])
spinB7_action = AnimationAction(AnimationMixer(proStackB7), spinB7_clip, proStackB7)

spinB8_track = bTrack
spinB8_clip = AnimationClip(tracks=[spinB8_track])
spinB8_action = AnimationAction(AnimationMixer(proStackB8), spinB8_clip, proStackB8)

spinB9_track = bTrack
spinB9_clip = AnimationClip(tracks=[spinB9_track])
spinB9_action = AnimationAction(AnimationMixer(proStackB9), spinB9_clip, proStackB9)

spinB10_track = bTrack
spinB10_clip = AnimationClip(tracks=[spinB10_track])
spinB10_action = AnimationAction(AnimationMixer(proStackB10), spinB10_clip, proStackB10)

spinB11_track = bTrack
spinB11_clip = AnimationClip(tracks=[spinB11_track])
spinB11_action = AnimationAction(AnimationMixer(proStackB11), spinB11_clip, proStackB11)

spinB12_track = bTrack
spinB12_clip = AnimationClip(tracks=[spinB12_track])
spinB12_action = AnimationAction(AnimationMixer(proStackB12), spinB12_clip, proStackB12)

spinB13_track = bTrack
spinB13_clip = AnimationClip(tracks=[spinB13_track])
spinB13_action = AnimationAction(AnimationMixer(proStackB13), spinB13_clip, proStackB13)

spinB14_track = bTrack
spinB14_clip = AnimationClip(tracks=[spinB14_track])
spinB14_action = AnimationAction(AnimationMixer(proStackB14), spinB14_clip, proStackB14)

spinB15_track = bTrack
spinB15_clip = AnimationClip(tracks=[spinB15_track])
spinB15_action = AnimationAction(AnimationMixer(proStackB15), spinB15_clip, proStackB15)

spinB16_track = bTrack
spinB16_clip = AnimationClip(tracks=[spinB16_track])
spinB16_action = AnimationAction(AnimationMixer(proStackB16), spinB16_clip, proStackB16)

spinB17_track = bTrack
spinB17_clip = AnimationClip(tracks=[spinB17_track])
spinB17_action = AnimationAction(AnimationMixer(proStackB17), spinB17_clip, proStackB17)

spinB18_track = bTrack
spinB18_clip = AnimationClip(tracks=[spinB18_track])
spinB18_action = AnimationAction(AnimationMixer(proStackB18), spinB18_clip, proStackB18)

spinB19_track = bTrack
spinB19_clip = AnimationClip(tracks=[spinB19_track])
spinB19_action = AnimationAction(AnimationMixer(proStackB19), spinB19_clip, proStackB19)

spinB20_track = aTrack
spinB20_clip = AnimationClip(tracks=[spinB20_track])
spinB20_action = AnimationAction(AnimationMixer(grpB1), spinB20_clip, grpB1)

# Layout the 19 rotation controls
rgridB = GridspecLayout(8, 7)

rgridB[1, 3] = spinB14_action
rgridB[2, 3] = spinB4_action
rgridB[2, 4] = spinB9_action
rgridB[2, 5] = spinB13_action
rgridB[3, 2] = spinB18_action
rgridB[3, 3] = spinB2_action
rgridB[3, 4] = spinB16_action
rgridB[4, 1] = spinB10_action
rgridB[4, 2] = spinB6_action
rgridB[4, 3] = spinB1_action
rgridB[4, 4] = spinB7_action
rgridB[4, 5] = spinB12_action
rgridB[5, 2] = spinB17_action
rgridB[5, 3] = spinB3_action
rgridB[5, 4] = spinB19_action
rgridB[6, 1] = spinB11_action
rgridB[6, 2] = spinB8_action
rgridB[6, 3] = spinB5_action
rgridB[7, 3] = spinB15_action
#rgridB[7, 5] = spinB20_action

In [18]:
# The widget tab enclosure.

# Including a non-interactive periodic table showing the atom selected.
b1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
b2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
b3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
b4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
b5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
b6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
b7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
b8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
b9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
b10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
b11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
b12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
b13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
b14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
b15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
b16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
b17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
b18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
b19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
b20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
b21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
b22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
b23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
b24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
b25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
b26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
b27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
b28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
b29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
b30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
b31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
b32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
b33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
b34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
b35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
b36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
b37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
b38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
b39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
b40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
b41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
b42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
b43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
b44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
b45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
b46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
b47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
b48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
b49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
b50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
b51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
b52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
b53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
b54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
b55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
b56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
b57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
b58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
b59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
b60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
b61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
b62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
b63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
b64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
b65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
b66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
b67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
b68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
b69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
b70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
b71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
b72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
b73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
b74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
b75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
b76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
b77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
b78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
b79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
b80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
b81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
b82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
b83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
b84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
b85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
b86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
b87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
b88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
b89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
b90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
bbttns = [b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,b15,b16,b17,b18,b19,
               b20,b21,b22,b23,b24,b25,b26,b27,b28,b29,b30,b31,b32,b33,b34,b35,b36,b37,b38,b39,
               b40,b41,b42,b43,b44,b45,b46,b47,b48,b49,b50,b51,b52,b53,b54,b55,b56,b57,b58,b59,
               b60,b61,b62,b63,b64,b65,b66,b67,b68,b69,b70,b71,b72,b73,b74,b75,b76,b77,b78,b79,
               b80,b81,b82,b83,b84,b85,b86,b87,b88,b89,b90]

bbttns[atomicNumber-1] = Button(description=dlist[atomicNumber-1], style=dict(button_color='white',font_weight='bold'))

# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    if i == atomicNumber-1 :
        bbttns[atomicNumber-1].layout = widgets.Layout(border = '2px solid red')
    else :
        bbttns[i].layout = widgets.Layout(border = '1px solid black')
    bbttns[i].layout.width = '100%'
    bbttns[i].layout.height = '100%'  #'40px'
    i += 1
    
# Layout the 90 buttons in periodic table form
gridb = GridspecLayout(10, 18)

gridb[0,0] = bbttns[0]
gridb[0,17] = bbttns[1]
gridb[1,0] = bbttns[2]
gridb[1,1] = bbttns[3]
p=12
while p < 18:
    gridb[1,p] = bbttns[p-8]
    p += 1
gridb[2,0] = bbttns[10]
gridb[2,1] = bbttns[11]
n=12
while n < 18:
    gridb[2,n] = bbttns[n]
    n += 1
m=0
while m < 18:
    gridb[3,m] = bbttns[18+m]
    m += 1
k=0
while k < 18:
    gridb[4,k] = bbttns[36+k]
    k += 1
gridb[5,0] = bbttns[54]
gridb[5,1] = bbttns[55]
# gridb[5,2] = the Lanthanide series hole
i=3
while i < 18:
    gridb[5,i] = bbttns[68+i]
    i += 1
gridb[6,0] = bbttns[86]
gridb[6,1] = bbttns[87]
# gridb[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    gridb[7,j] = bbttns[54+j]
    j += 1
#Actinide series
gridb[8,2] = bbttns[88]
gridb[8,3] = bbttns[89]

# The tab enclosure 
tab1 = VBox(children=[renderer]) # 3D Atom
tab2 = VBox(children=[sgrid])  # Slotlayout (SL) diagram
tab3 = VBox(children=[sngrid]) # SL w #'s of' left/right neutrons
tab4 = VBox(children=[gridb])  # Periodic table

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, atomicLabel)
tab.set_title(1, dlist[number-1]+' Slotlayout (SL)')
tab.set_title(2, dlist[number-1]+' SL 1.TBA 5.RLAB')
tab.set_title(3, 'Periodic Table')

#VBox(children=[tab]) # Display a tab enclosure with all outputs.
# Displayed between the top (R) and bottom (L) rotation controls.

In [19]:
"""Rotation controls for all slots' right spinning proton groups.
The Carousel's R spin control is at the bottom right. 
Less than half the slots conbtain only right spin groups and about as many 
other slots contain left spin groups. Some slots, i.e. slot1, contain both"""
rgridA  

GridspecLayout(children=(AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', t…

In [20]:
VBox(children=[tab]) # Display a tab enclosure with all outputs

In [21]:
rgridB  # Rotation controls for just the left spinning proton groups in all slots.

GridspecLayout(children=(AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', t…


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).

**SECTION 9: THE NUCLEUS** contains descriptions and diagrams 
of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project greatly relies.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'll try to figure it out.

Please pardon my understandings, personal interpretations and mistakes. Feel free to make this project better.